### Pandas Dataframe to JSON
Here, we test and demonstrate how to get data from the back-end to the front via json.

First we create a dataframe of fake data for three stations and three variables. Some variables are missing for some stations. We want to see how we handle missing values and varying column/variable names. We will use these three stations, eventually switching for real data:

| Location   | Variable 1  | Variable 2 | Variable 3   |
|------------|-------------|------------|--------------|
| Roosevelt  | PM2.5       | Ozone      | Temperature  |
| Vernal     | Ozone       | NOx        | PM2.5        |
| Horsepool  | NO          | NOx        | Temperature  |



In [3]:


import os 

import pandas as pd 
import numpy as np 



In [4]:
# Create a dataframe that holds the test data for the three stations of interest. 
# The columns are the variables of interest and each station only has a max 
# of three variables, hence some columns are full of NaN values for some stations.

# Example numbers 
# NO 3 ppb
# NOx 4 ppb
# Temp 21 C 
# Wind Speed 2 m/s
# PM2.5 4 ug/m3
# Ozone 40 ppb


df_dict = {
    "Roosevelt": {
        "PM2.5": 5.0,
        "Ozone": 46.0,
        "Temperature": 16.5,
            },
    "Vernal": {
        "Ozone": 42.0,
        "NOx": 4.3,
        "PM2.5": 2.5,
            },
    "Horsepool": {
        "NO": 3.0,
        "NOx": 4.0,
        "Temperature": 18.0,
            }
}

df = pd.DataFrame(df_dict)
df

,Roosevelt,Vernal,Horsepool
PM2.5,5.0,2.5,NaN
Ozone,46.0,42.0,NaN
Temperature,16.5,NaN,18.0
NOx,NaN,4.3,4.0
NO,NaN,NaN,3.0


In [6]:
# Convert the dataframe to a JSON string
json_str = df.to_json(orient="index")

# Print the JSON string to preview it
print(json_str)

{"PM2.5":{"Roosevelt":5.0,"Vernal":2.5,"Horsepool":null},"Ozone":{"Roosevelt":46.0,"Vernal":42.0,"Horsepool":null},"Temperature":{"Roosevelt":16.5,"Vernal":null,"Horsepool":18.0},"NOx":{"Roosevelt":null,"Vernal":4.3,"Horsepool":4.0},"NO":{"Roosevelt":null,"Vernal":null,"Horsepool":3.0}}


In [ ]:
# Save the JSON string to a file in the test_data directory based at the root 
# This notebook is in backend, so parallel (up and down) to test_data directory
fpath = os.path.join("..", "test_data", "test_data.json")
with open(fpath, "w") as f:
    f.write(json_str)